In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the train and test data
train_df = pd.read_csv("../data/ESI-handbook/Train.csv")
test_df  = pd.read_csv("../data/ESI-handbook/test.csv")

In [4]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Clinical Vignettes  209 non-null    object
 1   acuity              209 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.4+ KB


In [23]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.neural_network import MLPClassifier
import sys
sys.path.append('..')
from utils import utils

# Load and initialize the embedding model
model_name = 'pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb'
model = SentenceTransformer(model_name)

# Create embeddings for train and test data
train_embeddings = model.encode(train_df['Clinical Vignettes'].values)
test_embeddings = model.encode(test_df['Clinical Vignettes'].values)

# Save embeddings
np.save('../data/ESI-handbook/train_embeddings.npy', train_embeddings)
np.save('../data/ESI-handbook/test_embeddings.npy', test_embeddings)

# Get labels
train_labels = train_df['acuity'].values
test_labels = test_df['acuity'].values

# Train MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(250,100), early_stopping=True)
mlp.fit(train_embeddings, train_labels)

# Make predictions
predictions = mlp.predict(test_embeddings)

# Calculate and save metrics
metrics = utils.evaluate_predictions(predictions, test_labels, ordinal=True, by_class=True)
print("Overall Metrics:", metrics)
output_filepath = "../results/Triage-Handbook/Triage-Handbook_BioBERT"
utils.save_metrics(metrics, output_filepath)
print("Evaluation complete. Metrics and plots saved.")


Overall Metrics: {'overall': {'accuracy': 0.28708133971291866, 'precision': 0.24729239238104986, 'recall': 0.28708133971291866, 'f1_score': 0.25113503765077716, 'adjusted_accuracy': 0.645933014354067, 'adjusted_precision': 0.74775774042867, 'adjusted_recall': 0.645933014354067, 'adjusted_f1': 0.5845436114165596, 'mae': 1.2870813397129186, 'mse': 2.9043062200956937, 'quadratic_kappa': np.float64(0.11741952539637279)}, 'by_class': {'1': {'precision': 0.18181818181818182, 'recall': 0.16666666666666666, 'f1-score': 0.17391304347826086, 'support': 36.0}, '2': {'precision': 0.2773109243697479, 'recall': 0.55, 'f1-score': 0.3687150837988827, 'support': 60.0}, '3': {'precision': 0.09090909090909091, 'recall': 0.09090909090909091, 'f1-score': 0.09090909090909091, 'support': 33.0}, '4': {'precision': 0.75, 'recall': 0.5294117647058824, 'f1-score': 0.6206896551724138, 'support': 34.0}, '5': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 46.0}, 'accuracy': 0.28708133971291866, 'macr

/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera